## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-07-16-55-38 +0000,nypost,NYC guys are splurging on marathon dates as lo...,https://nypost.com/2025/08/07/lifestyle/nyc-gu...
1,2025-08-07-16-54-00 +0000,wsj,Netanyahu Says Israel Will Take Over the Entir...,https://www.wsj.com/world/middle-east/netanyah...
2,2025-08-07-16-52-30 +0000,bbc,David Lammy to host JD Vance at country residence,https://www.bbc.com/news/articles/cgjylqlvq5xo...
3,2025-08-07-16-51-52 +0000,nyt,Netanyahu Says Israel Wants to Take Military C...,https://www.nytimes.com/2025/08/07/world/middl...
4,2025-08-07-16-50-33 +0000,bbc,Texas redistricting feud escalates as Democrat...,https://www.bbc.com/news/articles/c1jn5wpyg1jo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,80
145,tariffs,24
48,new,17
210,putin,16
236,india,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
281,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...,147
138,2025-08-07-10-35-00 +0000,nypost,Trump will only meet with Putin if Russian pre...,https://nypost.com/2025/08/07/us-news/trump-to...,136
21,2025-08-07-16-12-12 +0000,wapo,Trump celebrates new tariffs as negotiations c...,https://www.washingtonpost.com/business/2025/0...,135
248,2025-08-06-21-56-54 +0000,wapo,"Trump raises tariffs on India to 50 percent, c...",https://www.washingtonpost.com/business/2025/0...,134
118,2025-08-07-11-54-12 +0000,bbc,Trump's sweeping new tariffs take effect again...,https://www.bbc.com/news/articles/cx23jmvn5yzo...,133


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
281,147,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...
193,82,2025-08-07-04-12-00 +0000,wsj,"China’s exports grew at a faster clip in July,...",https://www.wsj.com/economy/trade/chinas-expor...
25,72,2025-08-07-16-04-00 +0000,wsj,Sen. John Cornyn said the FBI will help track ...,https://www.wsj.com/politics/policy/fbi-texas-...
240,54,2025-08-06-22-24-32 +0000,nypost,GOP Sen. Tom Cotton questions new Intel CEO Li...,https://nypost.com/2025/08/06/business/gop-sen...
94,42,2025-08-07-13-13-58 +0000,bbc,"First migrants detained under 'one in, one out...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
168,39,2025-08-07-09-00-15 +0000,wapo,"In tariff spat, Trump wields India-Russia trad...",https://www.washingtonpost.com/world/2025/08/0...
217,37,2025-08-07-00-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
282,34,2025-08-06-19-42-41 +0000,latimes,Sea lion bites young surfer in Monterey County...,https://www.latimes.com/california/story/2025-...
15,34,2025-08-07-16-23-38 +0000,nypost,Trump critic Rep. Jasmine Crockett is a no-sho...,https://nypost.com/2025/08/07/us-news/rep-jasm...
113,32,2025-08-07-12-04-00 +0000,nypost,"Vance Boelter, man charged with killing top Mi...",https://nypost.com/2025/08/07/us-news/man-char...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
